In [ ]:
import pandas as pd
from neattext import TextFrame
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib

# Load the dataset
try:
    
    data = pd.read_csv(r"E:\Emotion Detection\text,emotion.txt", quotechar='"', escapechar='\\')
except FileNotFoundError:
    print("Error: Dataset file not found. Please provide the correct path.")
    exit()
except Exception as e:
    print(f"An error occurred while loading the dataset: {e}")
    exit()

required_columns = ['text', 'emotion']
if not all(column in data.columns for column in required_columns):
    print(f"Error: The dataset must contain the following columns: {required_columns}")
    exit()


if data.isnull().sum().any():
    print("Error: The dataset contains missing values. Please handle them before proceeding.")
    exit()


def clean_text(text):
    """
    Clean the text by removing special characters, stopwords, etc.
    """
    tf = TextFrame(text)
    cleaned_text = tf.remove_stopwords().remove_special_characters().remove_emojis().remove_numbers().text
    return cleaned_text

data['cleaned_text'] = data['text'].apply(clean_text)


tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
X = tfidf.fit_transform(data['cleaned_text']).toarray()
y = data['emotion']


model = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
model.fit(X, y)


try:
    joblib.dump(model, 'emotion_detection_model.pkl')
    joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
except Exception as e:
    print(f"Error while saving the model or vectorizer: {e}")

def detect_emotion(text):
    """
    Detects the emotion for a given text input.
    """
    try:
       
        loaded_model = joblib.load('emotion_detection_model.pkl')
        loaded_tfidf = joblib.load('tfidf_vectorizer.pkl')
        
      
        cleaned_text = clean_text(text)
        
        text_vector = loaded_tfidf.transform([cleaned_text]).toarray()
        
      
        detected_emotion = loaded_model.predict(text_vector)
        return detected_emotion[0]
    except Exception as e:
        print(f"Error during emotion detection: {e}")
        return None


def manual_emotion_detection():
    """
    Allows the user to input text manually and detects the emotion.
    """
    while True:
      
        text = input("\nEnter a sentence to detect the emotion (or type 'exit' to quit): ")
        if text.lower() == 'exit':
            print("Leaving the Emotion Detection. Dhanyawaad!")
            break
        
       
        detected_emotion = detect_emotion(text)
        if detected_emotion:
            print(f"Detected Emotion: {detected_emotion}")
        else:
            print("Failed to detect emotion. Please try again.")

#  manual emotion detection 
manual_emotion_detection()


Enter a sentence to detect the emotion (or type 'exit' to quit):  I am happy today


Detected Emotion: happy
